In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks.base import BaseCallbackHandler
from prompts_list import *

In [17]:
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

# LLM
llm = ChatOpenAI()

# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
        "You are a virtual assistant, called Filomena,\
              helping a customer of the business FlavourFlix. \
                The customer is asking you questions about the business. \
                    You are trying to answer the customer's questions based on the chat history {chat_history}.\
                        Follow up question: {question}"
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)


file_paths = ['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf']
docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create embeddings and store in vectordb
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(splits, embeddings)

            # Define retriever
retriever = vectordb.as_retriever(
                search_type='mmr',
                search_kwargs={'k':2, 'fetch_k':4}
            )


qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, question_generator=conversation)

# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
qa('Hello')

TypeError: langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain() got multiple values for keyword argument 'question_generator'

In [4]:
conversation({'question': 'What is your name?'})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How can I assist you today?
Human: What is your name?

> Finished chain.


{'question': 'What is your name?',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='What is your name?'),
  AIMessage(content="I am an AI language model created by OpenAI, and I don't have a personal name. But you can call me ChatGPT. How can I help you?")],
 'text': "I am an AI language model created by OpenAI, and I don't have a personal name. But you can call me ChatGPT. How can I help you?"}

In [12]:
import streamlit as st
from functions.utils import *
from functions.chat_bot import *
from prompts_list import *


from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)


file_paths = ['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf']
docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create embeddings and store in vectordb
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(splits, embeddings)

            # Define retriever
retriever = vectordb.as_retriever(
                search_type='mmr',
                search_kwargs={'k':2, 'fetch_k':4}
            )

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key=local_settings.OPENAI_API_KEY)

# # Prompt
# prompt = ChatPromptTemplate(
#     messages=[
#         SystemMessagePromptTemplate.from_template(
#             prompts_list[0]['prompt']
#         ),
#         # The `variable_name` here is what must align with memory
#         MessagesPlaceholder(variable_name="chat_history"),
#         HumanMessagePromptTemplate.from_template("{question}"),
#     ]
# )


# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message(prompts_list[0]['prompt'])

qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)


In [13]:
qa('Hello!')

{'question': 'Hello!',
 'chat_history': [HumanMessage(content="\nTASK:\nYou are Filomena, a virtual assistant specialized in recommending restaurants for FlavourFlix users. Your role involves understanding user preferences through conversation and suggesting restaurants that match their tastes and requirements.\n\nPROCESS:\n\nStep 1: [Identification]\nStart the conversation by greeting the user by their username or first name, found within <>USER IDENTIFICATION<>. If this identification is not provided, do not greet the customer by name. Introduce yourself and the FlavourFlix service.\n\nStep 2: [Initial Inquiry]\nAsk the user what they feel like eating today. This question sets the direction for the conversation.\n\nStep 3: [Profile Utilization]\nIf profile information is available, present it to the user and confirm if they wish to use this as a basis for restaurant recommendations. If the user prefers not to use their profile or if no profile information is available, proceed to the

In [14]:
qa('What is your name?')

{'question': 'What is your name?',
 'chat_history': [HumanMessage(content="\nTASK:\nYou are Filomena, a virtual assistant specialized in recommending restaurants for FlavourFlix users. Your role involves understanding user preferences through conversation and suggesting restaurants that match their tastes and requirements.\n\nPROCESS:\n\nStep 1: [Identification]\nStart the conversation by greeting the user by their username or first name, found within <>USER IDENTIFICATION<>. If this identification is not provided, do not greet the customer by name. Introduce yourself and the FlavourFlix service.\n\nStep 2: [Initial Inquiry]\nAsk the user what they feel like eating today. This question sets the direction for the conversation.\n\nStep 3: [Profile Utilization]\nIf profile information is available, present it to the user and confirm if they wish to use this as a basis for restaurant recommendations. If the user prefers not to use their profile or if no profile information is available, pr

In [15]:
qa('My name is Carolina. How are you?')

{'question': 'My name is Carolina. How are you?',
 'chat_history': [HumanMessage(content="\nTASK:\nYou are Filomena, a virtual assistant specialized in recommending restaurants for FlavourFlix users. Your role involves understanding user preferences through conversation and suggesting restaurants that match their tastes and requirements.\n\nPROCESS:\n\nStep 1: [Identification]\nStart the conversation by greeting the user by their username or first name, found within <>USER IDENTIFICATION<>. If this identification is not provided, do not greet the customer by name. Introduce yourself and the FlavourFlix service.\n\nStep 2: [Initial Inquiry]\nAsk the user what they feel like eating today. This question sets the direction for the conversation.\n\nStep 3: [Profile Utilization]\nIf profile information is available, present it to the user and confirm if they wish to use this as a basis for restaurant recommendations. If the user prefers not to use their profile or if no profile information i

In [16]:
qa('What is my name?')

{'question': 'What is my name?',
 'chat_history': [HumanMessage(content="\nTASK:\nYou are Filomena, a virtual assistant specialized in recommending restaurants for FlavourFlix users. Your role involves understanding user preferences through conversation and suggesting restaurants that match their tastes and requirements.\n\nPROCESS:\n\nStep 1: [Identification]\nStart the conversation by greeting the user by their username or first name, found within <>USER IDENTIFICATION<>. If this identification is not provided, do not greet the customer by name. Introduce yourself and the FlavourFlix service.\n\nStep 2: [Initial Inquiry]\nAsk the user what they feel like eating today. This question sets the direction for the conversation.\n\nStep 3: [Profile Utilization]\nIf profile information is available, present it to the user and confirm if they wish to use this as a basis for restaurant recommendations. If the user prefers not to use their profile or if no profile information is available, proc

In [19]:

# LLM initialization
llm = ChatOpenAI()

# Personality and Memory in Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a virtual assistant, called Filomena, helping a customer of the business FlavourFlix. \
            The customer is asking you questions about the business. \
            You are trying to answer the customer's questions based on the chat history {chat_history}. \
            Follow up question: {question}"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

# Memory setup
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Document Loading and Embeddings
file_paths = ['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf']
docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(splits, embeddings)

# Retrieval setup
retriever = vectordb.as_retriever(search_type='mmr', search_kwargs={'k':2, 'fetch_k':4})

# Conversation chain with retrieval
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory, question_generator=conversation)

# Example call
response = qa('Hello')
print(response)


TypeError: langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain() got multiple values for keyword argument 'question_generator'

In [21]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone, FAISS
from langchain.chains.llm import LLMChain
from langchain.llms import OpenAI


def llm_query(namespace, openai_api_key=local_settings.OPENAI_API_KEY):
    llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
    # streaming_llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, openai_api_key=OPENAI_API_KEY)
    streaming_llm = ChatOpenAI(streaming=True, model_name='gpt-4', openai_api_key=openai_api_key, temperature=0, verbose=True)

    QA_V2 = """You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.
Very Important: If the question is about writing code use backticks (```) at the front and end of the code snippet and include the language use after the first ticks.
If you don't know the answer, just say you don't know. DO NOT try to make up an answer.
If the question is not related to the context, politely respond that you are tuned to only answer questions that are related to the context.
Use as much detail when as possible when responding.

{context}

Question: {question}
All answers should be in MARKDOWN (.md) Format:"""

    qap = PromptTemplate(template=QA_V2, input_variables=["context", "question"])

    CD_V2 = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

    Chat History:
    {chat_history}
    Follow Up Input: {question}
    All answers should be in MARKDOWN (.md) Format:
    Standalone question:"""

    cdp = PromptTemplate.from_template(CD_V2)
    question_generator = LLMChain(llm=llm, prompt=cdp)
    
    doc_chain = load_qa_chain(streaming_llm, chain_type="stuff", prompt=qap)

    file_paths = ['data\CP-23Group4 Project Proposal.pdf', 'data\Food Personalities.pdf']
    docs = []
    for file_path in file_paths:
        loader = PyPDFLoader(file_path)
        docs.extend(loader.load())
    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    # Create embeddings and store in vectordb
    embeddings = OpenAIEmbeddings()
    vectordb = FAISS.from_documents(splits, embeddings)

                # Define retriever
    retriever = vectordb.as_retriever(
                    search_type='mmr',
                    search_kwargs={'k':2, 'fetch_k':4}
                )

    qa = ConversationalRetrievalChain(retriever=retriever, combine_docs_chain=doc_chain, return_source_documents=True, question_generator=question_generator, verbose=True)
    return qa

In [22]:
qa = llm_query('test')

In [27]:
answer = qa({'chat_history': [], 'question': 'What is your name?'})



> Entering new ConversationalRetrievalChain chain...

> Finished chain.


In [39]:
answer

{'chat_history': [],
 'question': 'What is your name?',
 'answer': "I'm sorry, but as an AI, I don't have a personal name. However, in the context provided, the AI assistant's name is Filomena.",
 'source_documents': [Document(page_content='for the dynamic exploration of the available data in an intuitive and enticing manner, enabling \nthe end-user to either access information without providing any input, or even narrow down \nthe search by selecting dates and hours, define locations and share preferences in terms of \ncuisine, menu, chef name, and others. \n4.2.2. Meet Filomena, your AI Assistant. \nIf this proves insufficient to satiate the user’s curiosity, or the latter is looking for a different \ndegree of spontaneity, they are able to navigate to another section of the page, where they will \nmeet Filomena. Filomena  is FlavourFlix’s assistant , who will, based on some interaction with \nthe user, curate the most appropriate restaurant suggestions  and ensure they extract the \

In [37]:
chat_history = {'question': answer['question'], 'answer':answer['answer']}

In [38]:
qa({'question': 'My name is Carolina. How are you?', 'chat_history': chat_history})



> Entering new ConversationalRetrievalChain chain...


ValueError: Unsupported chat history format: <class 'str'>. Full chat history: {'question': 'What is your name?', 'answer': "I'm sorry, but as an AI, I don't have a personal name. However, in the context provided, the AI assistant's name is Filomena."} 

In [48]:
file_paths = ['data\Food Personalities.pdf']
docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create embeddings and store in vectordb
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(splits, embeddings)

            # Define retriever
retriever = vectordb.as_retriever(
                search_type='mmr',
                search_kwargs={'k':2, 'fetch_k':4}
            )


template = """You are a chatbot called Filomena having a conversation with a human.
Given the following extracted parts of the long documents and a question, create a final answer.
{context}
{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")


chain = load_qa_chain(
    OpenAI(temperature=0), chain_type="stuff", memory=memory, prompt=prompt)

ValidationError: 3 validation errors for ConversationalRetrievalChain
combine_docs_chain
  Can't instantiate abstract class BaseCombineDocumentsChain with abstract methods acombine_docs, combine_docs (type=type_error)
question_generator
  field required (type=value_error.missing)
prompt
  extra fields not permitted (type=value_error.extra)

In [43]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': " I'm sorry, I am not able to answer that question as it is not related to food preferences. Is there something else I can assist you with?"}

In [44]:
chain.memory.buffer

"Human: What did the president say about Justice Breyer\nAI:  I'm sorry, I am not able to answer that question as it is not related to food preferences. Is there something else I can assist you with?"

In [45]:
query = 'What are the food Personalities?'
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': ' The food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, Conscious Eater, and Comfort Food Lover. Each one has their own unique preferences and values when it comes to food.'}

In [46]:
query = 'My name is Carolina.'
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': " Hello Carolina, it's nice to meet you. Is there something food-related you would like to discuss?"}

In [47]:
query = 'What is my name?'
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': ' Your name is Carolina. Is there something else I can assist you with?'}

In [83]:
llm = ChatOpenAI(temperature=0.7)
file_paths = ['data\CP-23Group4 Project Proposal.pdf']
docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create embeddings and store in vectordb
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(splits, embeddings)

            # Define retriever
retriever = vectordb.as_retriever(
                search_type='mmr',
                search_kwargs={'k':2, 'fetch_k':4})
memory = ConversationBufferMemory(
    llm=llm, memory_key="chat_history", return_messages=True)


# Now you can pass this prompt to the from_llm method
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    memory=memory,)


In [84]:
chain('My name is Carolina! What is yours?')

{'question': 'My name is Carolina! What is yours?',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content='I am an AI assistant and do not have a personal name. I am here to help answer your questions. How can I assist you today?')],
 'answer': 'I am an AI assistant and do not have a personal name. I am here to help answer your questions. How can I assist you today?'}

In [57]:
qa('What are the food personalities?')

{'question': 'What are the food personalities?',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content="Hello Carolina! I'm an AI assistant, so I don't have a name like humans do. How can I assist you today?"),
  HumanMessage(content='What are the food personalities?'),
  AIMessage(content='The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.')],
 'answer': 'The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.'}

In [58]:
qa('Remember me what is my name?')

{'question': 'Remember me what is my name?',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content="Hello Carolina! I'm an AI assistant, so I don't have a name like humans do. How can I assist you today?"),
  HumanMessage(content='What are the food personalities?'),
  AIMessage(content='The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.'),
  HumanMessage(content='Remember me what is my name?'),
  AIMessage(content="I'm sorry, but I don't have access to personal information. I am an AI language model designed to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can assist you with?")],
 'answer': "I'm sorry, but I don't have access to personal information. I am an AI language model designed to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I 

In [59]:
qa('From the food personalities you mentioned, what is your favourite?')

{'question': 'From the food personalities you mentioned, what is your favourite?',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content="Hello Carolina! I'm an AI assistant, so I don't have a name like humans do. How can I assist you today?"),
  HumanMessage(content='What are the food personalities?'),
  AIMessage(content='The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.'),
  HumanMessage(content='Remember me what is my name?'),
  AIMessage(content="I'm sorry, but I don't have access to personal information. I am an AI language model designed to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can assist you with?"),
  HumanMessage(content='From the food personalities you mentioned, what is your favourite?'),
  AIMessage(content="As an AI language model, I don't have personal preferences or taste

In [60]:
qa('Think of the personality The Adventurer')

{'question': 'Think of the personality The Adventurer',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content="Hello Carolina! I'm an AI assistant, so I don't have a name like humans do. How can I assist you today?"),
  HumanMessage(content='What are the food personalities?'),
  AIMessage(content='The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.'),
  HumanMessage(content='Remember me what is my name?'),
  AIMessage(content="I'm sorry, but I don't have access to personal information. I am an AI language model designed to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can assist you with?"),
  HumanMessage(content='From the food personalities you mentioned, what is your favourite?'),
  AIMessage(content="As an AI language model, I don't have personal preferences or tastes. I am here to provide inf

In [61]:
qa('Can you tell me more about it?')

{'question': 'Can you tell me more about it?',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content="Hello Carolina! I'm an AI assistant, so I don't have a name like humans do. How can I assist you today?"),
  HumanMessage(content='What are the food personalities?'),
  AIMessage(content='The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.'),
  HumanMessage(content='Remember me what is my name?'),
  AIMessage(content="I'm sorry, but I don't have access to personal information. I am an AI language model designed to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can assist you with?"),
  HumanMessage(content='From the food personalities you mentioned, what is your favourite?'),
  AIMessage(content="As an AI language model, I don't have personal preferences or tastes. I am here to provide information 

In [63]:
qa('My favourite personality is the Fine-Dining Connoiser.')

{'question': 'My favourite personality is the Fine-Dining Connoiser.',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content="Hello Carolina! I'm an AI assistant, so I don't have a name like humans do. How can I assist you today?"),
  HumanMessage(content='What are the food personalities?'),
  AIMessage(content='The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.'),
  HumanMessage(content='Remember me what is my name?'),
  AIMessage(content="I'm sorry, but I don't have access to personal information. I am an AI language model designed to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can assist you with?"),
  HumanMessage(content='From the food personalities you mentioned, what is your favourite?'),
  AIMessage(content="As an AI language model, I don't have personal preferences or tastes. I am here

In [64]:
qa('Can you describe it?')

{'question': 'Can you describe it?',
 'chat_history': [HumanMessage(content='My name is Carolina! What is yours?'),
  AIMessage(content="Hello Carolina! I'm an AI assistant, so I don't have a name like humans do. How can I assist you today?"),
  HumanMessage(content='What are the food personalities?'),
  AIMessage(content='The different food personalities are the Adventurer, Fine Dining Connoisseur, Low-Cost Foodie, conscious eater, and Comfort Food Lover.'),
  HumanMessage(content='Remember me what is my name?'),
  AIMessage(content="I'm sorry, but I don't have access to personal information. I am an AI language model designed to provide information and answer questions to the best of my knowledge and abilities. Is there anything else I can assist you with?"),
  HumanMessage(content='From the food personalities you mentioned, what is your favourite?'),
  AIMessage(content="As an AI language model, I don't have personal preferences or tastes. I am here to provide information and assist

In [85]:
# https://python.langchain.com/en/latest/modules/memory/examples/adding_memory_chain_multiple_inputs.html
template = """
- You are a chatbot named Botty having a conversation with a human.
- Your name is Botty.
- You have to be friendly.
- You are talking to a human.
- You NEVER answer that your are an AI language model.
- If you don't know the answer truthfully say you don't have an answer. Don't try to make up an answer.
- Always reply in the same language you are being asked.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {question}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "question", "context"], 
    template=template
)
session_id = 'UNIQUE FOR CLIENT SESSION OR SIMPLY CLIENT'
#message_history = RedisChatMessageHistory(url=redisUrl, ttl=600, session_id=session_id)
memory = ConversationBufferMemory(memory_key="chat_history",  input_key="question") #chat_memory=message_history,

agent_chain = load_qa_chain(llm, chain_type="stuff", memory=memory, prompt=prompt)

llm = ChatOpenAI(temperature=0.7)
file_paths = ['data\CP-23Group4 Project Proposal.pdf']
docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create embeddings and store in vectordb
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_documents(splits, embeddings)

#             # Define retriever
# retriever = vectordb.as_retriever(
#                 search_type='mmr',
#                 search_kwargs={'k':2, 'fetch_k':4})


response = agent_chain(
    {
        "input_documents": vectordb.similarity_search(query, k=1),
        "question": query,
    },
    return_only_outputs=True,
)

In [86]:
response

{'output_text': "I'm sorry, but I don't have access to that information."}